In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Any
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import uvicorn
from itertools import cycle, islice
import traceback
import numpy as np

In [ ]:
import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()
ngrok.set_auth_token("2wcLhKAkhn0MwFEwOlt2CXaabqk_4n42xCLtLqGuFKiKLfrik")

In [ ]:
# ---------------------- 예시 운동 데이터 ----------------------
exercise_data = [
    {"id": "ex001", "name": "푸쉬업", "target_parts": ["가슴", "전신"], "equipment": ["맨몸", "덤벨"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 2, "calories_per_min": 8, "youtube": "https://www.youtube.com/watch?v=IODxDxX7oi4"},
    {"id": "ex002", "name": "크런치", "target_parts": ["복부"], "equipment": ["맨몸"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 1, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=Xyd_fa5zoEU"},
    {"id": "ex003", "name": "스쿼트", "target_parts": ["하체"], "equipment": ["맨몸", "바벨"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 3, "calories_per_min": 7, "youtube": "https://www.youtube.com/watch?v=aclHkVaku9U"},
    {"id": "ex004", "name": "버피", "target_parts": ["전신"], "equipment": ["맨몸"], "level": "advanced", "difficulty_score": 3, "type": "HIIT", "avg_duration": 1, "calories_per_min": 12, "youtube": "https://www.youtube.com/watch?v=dZgVxmf6jkA"},
    {"id": "ex005", "name": "런지", "target_parts": ["하체"], "equipment": ["맨몸", "덤벨"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 2, "calories_per_min": 6, "youtube": "https://www.youtube.com/watch?v=QOVaHwm-Q6U"},
    {"id": "ex006", "name": "플랭크", "target_parts": ["복부"], "equipment": ["맨몸"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 2, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=pSHjTRCQxIw"},
    {"id": "ex007", "name": "자전거 크런치", "target_parts": ["복부"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "cardio", "avg_duration": 2, "calories_per_min": 9, "youtube": "https://www.youtube.com/watch?v=9FGilxCbdz8"},
    {"id": "ex008", "name": "마운틴 클라이머", "target_parts": ["전신"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "HIIT", "avg_duration": 1, "calories_per_min": 10, "youtube": "https://www.youtube.com/watch?v=nmwgirgXLYM"},
    {"id": "ex009", "name": "점핑잭", "target_parts": ["전신"], "equipment": ["맨몸"], "level": "beginner", "difficulty_score": 1, "type": "cardio", "avg_duration": 1, "calories_per_min": 7, "youtube": "https://www.youtube.com/watch?v=c4DAnQ6DtF8"},
    {"id": "ex010", "name": "스텝업", "target_parts": ["하체"], "equipment": ["맨몸", "스텝박스"], "level": "beginner", "difficulty_score": 1, "type": "cardio", "avg_duration": 2, "calories_per_min": 6, "youtube": "https://www.youtube.com/watch?v=dQqApCGd5Ss"},
    {"id": "ex011", "name": "트라이셉스 딥스", "target_parts": ["팔"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 2, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=tKjcgfu44sI"},
    {"id": "ex012", "name": "벤치 프레스", "target_parts": ["가슴"], "equipment": ["바벨"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 3, "calories_per_min": 8, "youtube": "https://www.youtube.com/watch?v=rT7DgCr-3pg"},
    {"id": "ex013", "name": "레그 프레스", "target_parts": ["하체"], "equipment": ["머신"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 3, "calories_per_min": 7, "youtube": "https://www.youtube.com/watch?v=IZxyjW7MPJQ"},
    {"id": "ex014", "name": "풀업", "target_parts": ["등"], "equipment": ["맨몸"], "level": "advanced", "difficulty_score": 3, "type": "strength", "avg_duration": 2, "calories_per_min": 9, "youtube": "https://www.youtube.com/watch?v=eGo4IYlbE5g"},
    {"id": "ex015", "name": "바벨 로우", "target_parts": ["등"], "equipment": ["바벨"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 3, "calories_per_min": 8, "youtube": "https://www.youtube.com/watch?v=vT2GjY_Umpw"},
    {"id": "ex016", "name": "사이드 런지", "target_parts": ["하체"], "equipment": ["맨몸", "덤벨"], "level": "beginner", "difficulty_score": 1, "type": "cardio", "avg_duration": 2, "calories_per_min": 7, "youtube": "https://www.youtube.com/watch?v=rvdv4WfIQJ0"},
    {"id": "ex017", "name": "킥백", "target_parts": ["하체"], "equipment": ["맨몸"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 1, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=1yHqLNL6K_A"},
    {"id": "ex018", "name": "벤치 딥스", "target_parts": ["팔"], "equipment": ["맨몸"] , "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 1, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=6kALZikXxLc"},
    {"id": "ex019", "name": "점프 스쿼트", "target_parts": ["하체", "전신"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "HIIT", "avg_duration": 1, "calories_per_min": 10, "youtube": "https://www.youtube.com/watch?v=U4s4mEQ5VqU"},
    {"id": "ex020", "name": "케틀벨 스윙", "target_parts": ["하체", "전신"], "equipment": ["케틀벨"], "level": "intermediate", "difficulty_score": 2, "type": "cardio", "avg_duration": 2, "calories_per_min": 9, "youtube": "https://www.youtube.com/watch?v=YSx3nFzxZBg"},
    {"id": "ex021", "name": "로잉 머신", "target_parts": ["전신"], "equipment": ["머신"], "level": "intermediate", "difficulty_score": 2, "type": "cardio", "avg_duration": 5, "calories_per_min": 11, "youtube": "https://www.youtube.com/watch?v=Ivuy8VWB3xA"},
    {"id": "ex022", "name": "스텝 점프", "target_parts": ["하체"], "equipment": ["스텝박스"], "level": "intermediate", "difficulty_score": 2, "type": "HIIT", "avg_duration": 2, "calories_per_min": 9, "youtube": "https://www.youtube.com/watch?v=aVUx3Et8aks"},
    {"id": "ex023", "name": "덤벨 컬", "target_parts": ["팔"], "equipment": ["덤벨"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 2, "calories_per_min": 4, "youtube": "https://www.youtube.com/watch?v=ykJmrZ5v0Oo"},
    {"id": "ex024", "name": "스파이더 크롤", "target_parts": ["전신"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "HIIT", "avg_duration": 2, "calories_per_min": 10, "youtube": "https://www.youtube.com/watch?v=w2TQ0d0LxP0"},
    {"id": "ex025", "name": "로프 스매시", "target_parts": ["전신"], "equipment": ["배틀로프"], "level": "advanced", "difficulty_score": 3, "type": "HIIT", "avg_duration": 1, "calories_per_min": 12, "youtube": "https://www.youtube.com/watch?v=ZoUoDE7H9Oo"},
    {"id": "ex026", "name": "체스트 플라이", "target_parts": ["가슴"], "equipment": ["머신"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 2, "calories_per_min": 6, "youtube": "https://www.youtube.com/watch?v=eozdVDA78K0"},
    {"id": "ex027", "name": "레그 컬", "target_parts": ["하체"], "equipment": ["머신"], "level": "beginner", "difficulty_score": 1, "type": "strength", "avg_duration": 2, "calories_per_min": 5, "youtube": "https://www.youtube.com/watch?v=1Tq3QdYUuHs"},
    {"id": "ex028", "name": "케이블 크런치", "target_parts": ["복부"], "equipment": ["케이블"], "level": "intermediate", "difficulty_score": 2, "type": "strength", "avg_duration": 2, "calories_per_min": 6, "youtube": "https://www.youtube.com/watch?v=U3HlEF_E9fo"},
    {"id": "ex029", "name": "스탠딩 바이시클 크런치", "target_parts": ["복부"], "equipment": ["맨몸"], "level": "beginner", "difficulty_score": 1, "type": "cardio", "avg_duration": 2, "calories_per_min": 8, "youtube": "https://www.youtube.com/watch?v=z6PJMT2y8GQ"},
    {"id": "ex030", "name": "하이 니즈", "target_parts": ["전신"], "equipment": ["맨몸"], "level": "intermediate", "difficulty_score": 2, "type": "HIIT", "avg_duration": 1, "calories_per_min": 11, "youtube": "https://www.youtube.com/watch?v=8opcQdC-V-U"}
]

# 목표별 세트×반복 매핑
rep_ranges = {
    "muscle_gain": "4x8",
    "fat_loss": "3x15",
    "maintenance": "3x12"
}

# 목표별 추천 타입 매핑
goal_to_types = {
    "muscle_gain": ["strength"],
    "fat_loss": ["cardio", "HIIT"],
    "maintenance": ["strength", "cardio", "HIIT"]
}

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-53' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [ ]:
# ------------------ 데이터프레임 및 벡터화 ------------------
df = pd.DataFrame(exercise_data)
# features: target_parts + equipment + level + type + difficulty_score + avg_duration + calories_per_min
def make_feature(r):
    parts = " ".join(r["target_parts"])
    equip = " ".join(r["equipment"])
    return f"{parts} {equip} {r['level']} {r['type']} score{r['difficulty_score']} dur{r['avg_duration']} cal{r['calories_per_min']}"
df["features"] = df.apply(make_feature, axis=1)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["features"])

In [ ]:
app = FastAPI(title="MVP Enhanced Content-Based Workout Recommender")

class RecommendationRequest(BaseModel):
    goal: str
    preferred_parts: List[str]
    level: str
    equipment: str
    frequency_per_week: int
    top_k: int = 3

@app.post("/recommend", response_model=Dict[str, Any])
def recommend(req: RecommendationRequest):
    try:
        if req.frequency_per_week <= 0 or not req.preferred_parts:
            raise HTTPException(status_code=400, detail="부위와 주당 운동 일수는 1개 이상이어야 합니다.")
        if req.goal not in rep_ranges:
            raise HTTPException(status_code=400, detail="지원하지 않는 목표입니다.")

        days = req.frequency_per_week
        bins = [[part] for part in islice(cycle(req.preferred_parts), days)]

        queries = [f"{p} {req.equipment} {req.level} {t}" for p in req.preferred_parts for t in goal_to_types.get(req.goal, [])]
        user_vecs = vectorizer.transform(queries)
        user_profile = np.asarray(user_vecs.mean(axis=0))  # fix np.matrix issue

        sims = cosine_similarity(user_profile, X).flatten()
        df["score"] = sims

        schedule: Dict[str, Any] = {}
        for i, parts in enumerate(bins, start=1):
            recs = []
            for part in parts:
                try:
                    base_types = goal_to_types.get(req.goal, [])
                    fallback_types = base_types + ["strength"]  # fallback 추가

                    # 기본 필터
                    mask = (
                        df["target_parts"].apply(lambda x: isinstance(x, list) and part in x) &
                        df["equipment"].apply(lambda x: isinstance(x, list) and req.equipment in x) &
                        df["type"].isin(base_types)
                    )
                    candidates = df[mask].sort_values("score", ascending=False).drop_duplicates("name").head(req.top_k)

                    # 부족하면 fallback 타입 사용
                    if len(candidates) < req.top_k:
                        fallback_mask = (
                            df["target_parts"].apply(lambda x: isinstance(x, list) and part in x) &
                            df["equipment"].apply(lambda x: isinstance(x, list) and req.equipment in x) &
                            df["type"].isin(fallback_types)
                        )
                        extra = df[fallback_mask].sort_values("score", ascending=False).drop_duplicates("name")
                        extra = extra[~extra["name"].isin(candidates["name"])]
                        candidates = pd.concat([candidates, extra]).head(req.top_k)

                    if not candidates.empty:
                        for _, row in candidates.iterrows():
                            recs.append({
                                "id": row["id"],
                                "name": row["name"],
                                "target_parts": row["target_parts"],
                                "equipment": row["equipment"],
                                "level": row["level"],
                                "type": row["type"],
                                "youtube": row["youtube"],
                                "sets": rep_ranges[req.goal],
                                "score": float(row["score"])
                            })
                    else:
                        recs.append({"message": f"{part} 부위에 대한 추천 결과가 없습니다."})

                except Exception as e:
                    print(f"[ERROR: part {part}]", str(e))
                    traceback.print_exc()
                    recs.append({"message": f"{part} 부위 처리 중 오류 발생."})

            schedule[f"Day{i}"] = {"target_parts": parts, "exercises": recs}

        return {"schedule": schedule}

    except Exception as e:
        print("[SERVER ERROR]", str(e))
        traceback.print_exc()
        raise HTTPException(status_code=500, detail="Internal server error")

In [ ]:
public_url = ngrok.connect(8000)
print(f"Swagger UI: {public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=8000)

Swagger UI: NgrokTunnel: "https://2193-34-41-42-175.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [948]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     1.241.21.88:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     1.241.21.88:0 - "POST /recommend HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [948]


In [ ]:
{
  "goal": "fat_loss",
  "preferred_parts": ["하체", "전신"],
  "level": "beginner",
  "frequency_per_week": 3
}